# Q3: Unitary Synthesis [3 points]

In this problem, you'll implement a brute force, approximate, single-qubit unitary synthesis algorithm using the Clifford + T gate set. It has been broken down into small parts, so don't be intimidated by the number of subquestions.

## Part A: Hilbert-Schmidt distance

Our algorithm should guarantee the output circuit is close to the target unitary. This distance can be quantified using something called the Hilbert-Schmidt distance, where $n$ is the number of qubits:
$$HS(U_1, U_2) := \sqrt{1-\frac{| Tr(U_1^\dagger U_2)| ^2}{2^{2n}}}$$

This outputs a value between 0 and 1 (inclusive) that indicates how close the two untaries are. 0 means they are equal up to global phase and 1 means they are very not equal.

Is this distance function symmetric? i.e. is $HS(U_1, U_2)$ equal to $HS(U_2, U_1)$? Either prove it is symmetric or provide a counterexample.

## Part B: Implement the Hilbert-Schmidt distance function

Fill in the function below to compute the Hilbert-Schmidt (HS) distance.

In [ ]:
import numpy as np
from numpy.typing import NDArray
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

def get_unitary_from_circuit(circuit: QuantumCircuit):
  return Operator(circuit).data

def hilbert_schmidt(unitary1: NDArray, unitary2: NDArray) -> float:
  pass # TODO: write your code here. You may find some numpy methods to be particularly helpful.

# A test. Do not delete anything below this and make sure the output is included in your submission.
target_unitary = np.matrix([[-0.63093663-0.52401339j,  0.2720076 +0.50332971j],
       [ 0.42561408+0.38233701j,  0.36164925+0.73612559j]])
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.s(0)
circuit.t(0)

print(hilbert_schmidt(target_unitary, get_unitary_from_circuit(circuit)))

None


Is the unitary this circuit implements close to the target unitary?

## Part C: Implement unitary synthesis

Implement a function to synthesize a quantum circuit that is within epsilon of the target unitary, as measured by the HS distance.

Fill in the function below.

In [ ]:
# Your function should return a circuit with only these gates
cliffordt_gate_set_1q = ["h", "s", "t"]

"""
target_unitary: the unitary to implement
epsilon: a float between 0 and 1 (inclusive)

Returns a qiskit.QuantumCircuit object such that hilbert_schmidt(target_unitary, get_unitary_from_circuit(synthesize(target_unitary, epsilon))) <= epsilon is True.
"""
def synthesize(target_unitary: np.NDArray, epsilon: float) -> QuantumCircuit:
  # Hint: brute force search by enumerating circuits starting from the empty circuit, then all circuits with 1 gate, and so on.
  pass

# A test. Do not delete anything below this and make sure the output is included in your submission.
target_unitary = np.matrix([[-0.63093663-0.52401339j,  0.2720076 +0.50332971j],
       [ 0.42561408+0.38233701j,  0.36164925+0.73612559j]])
synthesized_circuit = synthesize(target_unitary, 0.5)
print(synthesized_circuit)
print(hilbert_schmidt(target_unitary, get_unitary_from_circuit(synthesized_circuit)) <= 0.5)

AttributeError: module 'numpy' has no attribute 'NDArray'

## Part D: Scalability

Sweep the following epsilon values and plot the time in seconds it takes to find a solution.

In [ ]:
target_unitary = np.matrix([[-0.63093663-0.52401339j,  0.2720076 +0.50332971j],
       [ 0.42561408+0.38233701j,  0.36164925+0.73612559j]])
epsilons = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

# TODO: write your code here and make sure the plot is included in the ouput

What does the plot tell you about how this algorithm scales?

## Part E: Why is unitary synthesis important?

Why do we even need to solve this problem? What is the significance of the Clifford + T gate set and what type of gate does it fail to support natively?